In [56]:
import pandas as pd
from openai import OpenAI
from google.colab import drive
import re
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
test_df = pd.read_csv('/content/test.csv')
test_df.head()

,question,answer
0,Kimberly bought x packages of cat food and 6 p...,8
1,Four students scored a total of 251 points in ...,54
2,"John decides to buy new phones for him, his 2 ...",1800
3,The vending machines sell chips for 40 cents a...,5
4,A herd consists of camels and dromedaries. The...,56


In [11]:
first_five_rows = test_df.head(5)


In [12]:
first_five_rows

,question,answer
0,Kimberly bought x packages of cat food and 6 p...,8
1,Four students scored a total of 251 points in ...,54
2,"John decides to buy new phones for him, his 2 ...",1800
3,The vending machines sell chips for 40 cents a...,5
4,A herd consists of camels and dromedaries. The...,56


In [14]:
client = OpenAI(api_key = 'sk-proj-kc7vqqZp4jHbh89EWC25T3BlbkFJRH74KKJyQKpYDijdJpsa')

def ft_response(user_input):

    response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal::9CBKKv54",
    messages=[
      {"role": "system", "content": "You are a math problem solver."},
      {"role": "user", "content": user_input}
    ]
    )
    return response.choices[0].message.content

In [35]:
def naive_response(user_input):
    user_input = user_input + " Just give me the answer."

    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a math problem solver."},
      {"role": "user", "content": user_input},
      {"role": "user", "content": "Just give me the number."}  # Adding the follow-up request
    ]
    )
    return response.choices[0].message.content

In [30]:
def evaluate_ft_responses(test_df):
    correct_num = 0
    for i, row in test_df.iterrows():
        expected_answer = str(row['answer']) # since the ft_response function returns a string
        question = row['question']
        response = ft_response(question)
        if expected_answer == response:
            correct_num += 1
    return correct_num / len(test_df)

In [86]:
def evaluate_naive_responses(test_df):
    correct_num = 0
    for i, row in test_df.iterrows():
        expected_answer = str(row['answer']) # since the naive_response function returns a string
        question = row['question']
        response = naive_response(question)
        # Grab the numerical value from the response since the response contains the reasoning
        pattern = r'\d+'
        numbers = re.findall(pattern, response)
        response = numbers[0]
        if expected_answer == response:
            correct_num += 1
    return correct_num / len(test_df)

In [31]:
print(evaluate_ft_responses(test_df))

0.7874015748031497


In [83]:
print(ft_response("Luke is buying fabric for new curtains. There are five windows. Each window is 35 inches wide and Luke needs to buy fabric equal to 2 times the total width of the windows. How much fabric should he buy?"))

18


In [87]:
print(evaluate_naive_responses(test_df))

0.16338582677165353


In [52]:
print(ft_response("Farmer Brown's farm is 200 acres, and Farmer Smith's farm is 100 acres more than twice that. How many acres do the x farms have, The total when we add the numbers together is 700. What is the value of unknown variable x?"))

2
